# Machines Manufacturing Captal Budgeting Model (Project 1)

This model is a capital budgeting model using concepts of Time Value of Money:

- [**Setup**](#Setup): Imported the numpy_financial library to calculate the NPV of the cash flows. Also imported dataclass and numpy as basic libraries.
    - Dataclass is the class that contains our main inputs
    - Numpy is for basic calculations
    - numpy_financial is for calculating NPV, as the npv() function in numpy is deprecated
- [**Inputs**](#Inputs): the model uses the below inputs to calculate the NPV of the project:
    - n_phones: number of phones manufactured by a machine
    - price_scrap: scrap price of a machine post working lifetime
    - price_phone: price of the phone
    - cost_machine_adv: equity investment per year, used to buy either machines or spend on advertisements
    - cogs_phone: variable cost of the phone
    - n_life: number of working years for a single machine
    - n_machines: number of machines bought by the company
    - d_1: initial demand of the phone
    - g_d: growth rate of demand
    - max_year: number of years over which the company forecasts the NPV

## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [80]:
from dataclasses import dataclass
import numpy as np
import numpy_financial as npf

## Inputs

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created and used in the output section.

In [92]:
@dataclass
class ModelInputs:
    n_phones: float = 100000
    price_scrap: float = 50000
    price_phone: float = 500
    cost_machine_adv: float = 1000000
    cogs_phone: float = 250
    n_life: int = 10
    n_machines: int = 5
    d_1: float = 100000
    g_d: float = 0.2
    max_year: float = 20
    interest: float = 0.05

## Outputs:

The Cash flow and calculated NPV are calculated here. The cash_flow_calc

In [95]:
def cash_flow_calc(n_phones, price_scrap, price_phone, cost_machine_adv, cogs_phone, n_life, n_machines,
                      d_1, g_d, max_year):
        demand, revenue, var_phone, cash_flow, scrap_array = ([] for i in range(5))

        i = 1
        # calculation for changing demands of phones
        for i in range(max_year+1):
            if i <= n_machines:
                demand.insert(i, d_1)
            else: # advertising starts here, so demand keeps increasing with the given growth rate
                val = d_1 * (1 + g_d) ** (i - n_machines)
                demand.insert(i, val)
                
        supply = 0
        
        for i, j in zip(range(max_year+1), range(len(demand)+1)):
            quantity = 0
            if i == 0 and j == 0:
                pass
            # revenue and variable cost calculation
            elif i < n_life and j < n_life:
                supply = n_phones * j
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity)
                var_phone.insert(i, cogs_phone * quantity)
            elif i%n_machines == 0 and i == n_life and j == n_life:
                supply = n_phones * n_machines
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity)
                var_phone.insert(i, cogs_phone * quantity)
            elif i%n_machines == 0 and i-n_life == n_machines:
                supply = 0
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity)
                var_phone.insert(i, cogs_phone * quantity)
            elif i%n_machines == 0 and i > n_life and i - n_life > n_machines:
                supply = 0
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity) 
                var_phone.insert(i, cogs_phone * quantity)
            elif i%n_machines != 0 and i%n_life > n_machines:
                supply = 0
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity)
                var_phone.insert(i, cogs_phone * quantity)
            elif i%n_machines != 0 and i%n_life < n_machines:
                supply = n_phones * (n_machines - (i%n_life))
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity)
                var_phone.insert(i, cogs_phone * quantity)
            else:
                supply = n_phones * i
                quantity = min(demand[j], supply)
                revenue.insert(i, price_phone * quantity)
                var_phone.insert(i, cogs_phone * quantity)

            # scrap value calculation per year
            if i == 0 and j == 0:
                pass
            elif i > n_life and i - n_life <= n_machines:
                scrap_array.insert(i, price_scrap)
            else:
                scrap_array.insert(i, 0)

        # yearly cash flow calculation = Revenue cost of phone - variable cost - investment + scrap value
        for (i, j, k) in zip(range(len(revenue)), range(len(var_phone)), range(len(scrap_array))):
            calc = revenue[i]-var_phone[j]-cost_machine_adv+scrap_array[k]
            cash_flow.insert(i, calc)

        return cash_flow

def npv_calc(cash_flow, interest):
    npv = npf.npv(interest, cash_flow)
    return npv


model_data = ModelInputs()
cash_flows = cash_flow_calc(model_data.n_phones, model_data.price_scrap, model_data.price_phone, 
                    model_data.cost_machine_adv, model_data.cogs_phone, model_data.n_life, model_data.n_machines,
                    model_data.d_1, model_data.g_d, model_data.max_year)

print(f"Yearly Cash Flows: {cash_flows}")
npv = npv_calc(cash_flows, model_data.interest)
print(f"Net Present Value of Cash Flows: {npv}")

# cash_flows = # this should ultimately be set to the list containing your cash flow numbers in each year
# npv = # this should ultimately be set to the overally model npv number 

Yearly Cash Flows: [24000000, 24000000, 24000000, 24000000, 24000000, 29000000.0, 35000000.0, 42199999.99999999, 50840000.0, 61207999.999999985, 73699599.99999997, 74050000, 49050000, 24050000, -950000, -1000000, -1000000, -1000000, -1000000, -1000000]
Net Present Value of Cash Flows: 387740369.5978611
